In [282]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import MultiLabelBinarizer
import seaborn as sns
import os
import re
import spacy
from datetime import datetime, date
nlp = spacy.load('en')
from collections import Counter
%matplotlib inline

In [411]:
list_df = pd.read_csv('./archive/listings.csv')
cal_df = pd.read_csv('./archive/calendar.csv')
review_df = pd.read_csv('./archive/reviews.csv')

We want to determine what features a listing must have to generate the highest price point. Features examined will be neighbourhood, amenities included, and rating

- What style of accommodation  allows for highest revenue?
- Can we predict price using categorical and quantitative values?
- What effect does distance from city center have on price?

In [518]:
quant_cols = ['host_since', 'host_response_rate', 'host_acceptance_rate',
       'host_listings_count', 'host_total_listings_count', 'accommodates',
       'bathrooms', 'bedrooms', 'beds', 'square_feet', 'price', 'security_deposit',
       'cleaning_fee', 'guests_included', 'extra_people', 'minimum_nights',
       'maximum_nights', 'availability_30', 'availability_60', 'availability_90',
       'availability_365', 'number_of_reviews', 'first_review', 'last_review', 'review_scores_rating',
       'review_scores_accuracy', 'review_scores_cleanliness',
       'review_scores_checkin', 'review_scores_communication',
       'review_scores_location', 'review_scores_value', 'reviews_per_month']

cat_cols = ['host_response_time','host_is_superhost', 'host_verifications', 'host_has_profile_pic', 'host_identity_verified',
       'property_type', 'room_type', 'bed_type', 'amenities', 'instant_bookable',
       'cancellation_policy', 'require_guest_profile_picture', 'require_guest_phone_verification', 
       'neighbourhood_cleansed', 'city', 'state', 'country_code', 'requires_license', 'is_location_exact',
       'has_availability']

In [519]:
list_df[cat_cols].head()

,host_response_time,host_is_superhost,host_verifications,host_has_profile_pic,host_identity_verified,property_type,room_type,bed_type,amenities,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,neighbourhood_cleansed,city,state,country_code,requires_license,is_location_exact,has_availability
0,within a few hours,f,"['email', 'phone', 'reviews', 'kba']",t,t,Apartment,Entire home/apt,Real Bed,"{TV,""Cable TV"",Internet,""Wireless Internet"",""A...",f,moderate,f,f,West Queen Anne,Seattle,WA,US,f,t,t
1,within an hour,t,"['email', 'phone', 'facebook', 'linkedin', 're...",t,t,Apartment,Entire home/apt,Real Bed,"{TV,Internet,""Wireless Internet"",Kitchen,""Free...",f,strict,t,t,West Queen Anne,Seattle,WA,US,f,t,t
2,within a few hours,f,"['email', 'phone', 'google', 'reviews', 'jumio']",t,t,House,Entire home/apt,Real Bed,"{TV,""Cable TV"",Internet,""Wireless Internet"",""A...",f,strict,f,f,West Queen Anne,Seattle,WA,US,f,t,t
3,NaN,f,"['email', 'phone', 'facebook', 'reviews', 'jum...",t,t,Apartment,Entire home/apt,Real Bed,"{Internet,""Wireless Internet"",Kitchen,""Indoor ...",f,flexible,f,f,West Queen Anne,Seattle,WA,US,f,t,t
4,within an hour,f,"['email', 'phone', 'facebook', 'reviews', 'kba']",t,t,House,Entire home/apt,Real Bed,"{TV,""Cable TV"",Internet,""Wireless Internet"",Ki...",f,strict,f,f,West Queen Anne,Seattle,WA,US,f,t,t


### Quantitative Transformations

In [520]:
date_cols = ['host_since', 'first_review', 'last_review']
value_cols = ['host_response_rate', 'host_acceptance_rate',
              'price', 'extra_people', 'security_deposit', 'cleaning_fee']
num_cols = [i for i in quantitative if i not in date_cols+value_cols]

list_cols = ['host_verifications', 'amenities']

In [521]:
def clean_data(df, drop_var):
    
    df = df.dropna(subset=['price'], axis=0)
    
    for col in date_cols:
        for i in df[col].index:
            if not pd.isnull(df[col][i]):
                delta = date.today() - date.fromisoformat(df[col][i])
                df.loc[i, col] = delta.days
        df.loc[:, col] = df[col].astype(float)

    for col in value_cols:
        for i in df[col].index:
            if not pd.isnull(df[col][i]):
                df.loc[i, col] = re.sub("[$,%]", "", df[col][i])
        df.loc[:, col] = df[col].astype(float)

    for col in list_cols:
        for i in df[col].index:
            if not pd.isnull(df[col][i]):
                if '{' in df[col][i]:
                    df[col][i] = df[col][i].strip('{}').split(',')
                else:
                    df[col][i] = ast.literal_eval(df[col][i])
                
    for col in cat_cols:
        try:
            df = pd.concat([df.drop(col, axis=1), pd.get_dummies(df[col], 
                 prefix=col, prefix_sep='_', dummy_na=False, drop_first=True)], axis=1)
        except:
            df = df.drop(col, axis=1)
            continue

            
    num_vars = df.select_dtypes(include=['float', 'int']).columns
    nan_zero_cols = ['cleaning_fee', 'extra_people']
    
    for col in num_vars:
        if col in nan_zero_cols:
            df.loc[:, col].fillna(0, inplace=True)
        else:
            df.loc[:, col].fillna((df[col].mean()), inplace=True)
    
    y = df['price']
    df = df.drop(drop_var, axis=1)    
    X = df

    return X, y
    
#Use the function to create X and y
drop_list = ['id', 'listing_url', 'scrape_id', 'last_scraped', 'name', 'summary', 
             'experiences_offered', 'host_about','neighborhood_overview', 'space', 'price', 'description', 
             'notes', 'transit', 'thumbnail_url', 'medium_url', 'picture_url', 'xl_picture_url',
             'host_id', 'host_url', 'host_name', 'square_feet', 'host_thumbnail_url', 'host_picture_url',
             'zipcode', 'market', 'smart_location', 'country', 'latitude', 'longitude', 'weekly_price',
             'monthly_price', 'calendar_updated', 'calendar_last_scraped', 'license', 'jurisdiction_names', 'street',
             'neighbourhood', 'neighbourhood_group_cleansed', 'host_neighbourhood', 'host_location'] 

X, y = clean_data(list_df, drop_list)    

/Users/justinharrison/thesis/env/lib/python3.7/site-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/justinharrison/thesis/env/lib/python3.7/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [522]:
X.head()

,host_since,host_response_rate,host_acceptance_rate,host_listings_count,host_total_listings_count,accommodates,bathrooms,bedrooms,beds,security_deposit,...,neighbourhood_cleansed_Windermere,neighbourhood_cleansed_Yesler Terrace,city_Phinney Ridge Seattle,city_Seattle,city_Seattle,city_West Seattle,city_seattle,city_西雅图,state_wa,is_location_exact_t
0,3351.0,96.000000,100.000000,3.0,3.0,4,1.0,1.0,1.0,287.610397,...,0,0,0,1,0,0,0,0,0,1
1,2791.0,98.000000,100.000000,6.0,6.0,4,1.0,1.0,1.0,100.000000,...,0,0,0,1,0,0,0,0,0,1
2,2315.0,67.000000,100.000000,2.0,2.0,11,4.5,5.0,7.0,1000.000000,...,0,0,0,1,0,0,0,0,0,1
3,2533.0,94.886798,99.967159,1.0,1.0,3,1.0,0.0,2.0,287.610397,...,0,0,0,1,0,0,0,0,0,1
4,3241.0,100.000000,99.967159,2.0,2.0,6,2.0,3.0,3.0,700.000000,...,0,0,0,1,0,0,0,0,0,1


In [523]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .30, random_state=40) 

lm_model = LinearRegression(normalize=True) # Instantiate
lm_model.fit(X_train, y_train) #Fit
        
#Predict and score the model
y_test_preds = lm_model.predict(X_test) 
"The r-squared score for your model was {} on {} values.".format(r2_score(y_test, y_test_preds), len(y_test))

'The r-squared score for your model was -1.1935523632175966e+27 on 1146 values.'

In [524]:
def coef_weights(coefficients, X_train):
    '''
    INPUT:
    coefficients - the coefficients of the linear model 
    X_train - the training data, so the column names can be used
    OUTPUT:
    coefs_df - a dataframe holding the coefficient, estimate, and abs(estimate)
    
    Provides a dataframe that can be used to understand the most influential coefficients
    in a linear model by providing the coefficient estimates along with the name of the 
    variable attached to the coefficient.
    '''
    coefs_df = pd.DataFrame()
    coefs_df['est_int'] = X_train.columns
    coefs_df['coefs'] = lm_model.coef_
    coefs_df['abs_coefs'] = np.abs(lm_model.coef_)
    coefs_df = coefs_df.sort_values('abs_coefs', ascending=False)
    return coefs_df

#Use the function
coef_df = coef_weights(lm_model.coef_, X_train)

#A quick look at the top results
coef_df

,est_int,coefs,abs_coefs
150,city_Seattle,6.073103e+16,6.073103e+16
152,city_West Seattle,6.073103e+16,6.073103e+16
151,city_Seattle,6.073103e+16,6.073103e+16
153,city_seattle,3.464852e+16,3.464852e+16
155,state_wa,2.608251e+16,2.608251e+16
...,...,...,...
20,first_review,-1.022419e-02,1.022419e-02
0,host_since,-1.120979e-03,1.120979e-03
14,maximum_nights,5.315897e-04,5.315897e-04
149,city_Phinney Ridge Seattle,-4.974986e-08,4.974986e-08


In [506]:
with pd.option_context("display.max_rows", 1000):
    display(coef_df)

,est_int,coefs,abs_coefs
240,neighbourhood_North Beach/Blue Ridge,-3.653516e+16,3.653516e+16
96,host_neighbourhood_North Beach/Blue Ridge,3.653516e+16,3.653516e+16
47,host_neighbourhood_Clairemont Mesa East,-2.212986e+16,2.212986e+16
178,room_type_nan,1.951265e+16,1.951265e+16
362,city_Seattle,-1.942484e+16,1.942484e+16
364,city_West Seattle,-1.942484e+16,1.942484e+16
363,city_Seattle,-1.942484e+16,1.942484e+16
185,instant_bookable_nan,1.791424e+16,1.791424e+16
256,neighbourhood_Roxhill,-1.573797e+16,1.573797e+16
113,host_neighbourhood_Roxhill,1.573797e+16,1.573797e+16


In [208]:
ast.literal_eval(list_df['host_verifications'][0])

['email', 'phone', 'reviews', 'kba']

In [209]:
az_df = list_df['host_verifications']
for i in az_df.index:
    if not pd.isnull(az_df[i]):
        az_df[i] = ast.literal_eval(az_df[i])

/Users/justinharrison/thesis/env/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


['email', 'phone', 'reviews', 'kba']

In [213]:
new_df = list_df['host_verifications']
mlb = MultiLabelBinarizer()

pd.DataFrame(mlb.fit_transform(az_df),columns=mlb.classes_, index=az_df.index)

TypeError: 'NoneType' object is not iterable

In [214]:
pd.get_dummies(az_df.apply(pd.Series).stack()).sum(level=0)

,amex,email,facebook,google,jumio,kba,linkedin,manual_offline,manual_online,phone,photographer,reviews,sent_id,weibo
0,0,1,0,0,0,1,0,0,0,1,0,1,0,0
1,0,1,1,0,1,0,1,0,0,1,0,1,0,0
2,0,1,0,1,1,0,0,0,0,1,0,1,0,0
3,0,1,1,0,1,0,0,0,0,1,0,1,0,0
4,0,1,1,0,0,1,0,0,0,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3813,0,1,0,0,1,0,1,0,0,1,0,1,0,0
3814,0,1,1,0,1,0,0,0,0,1,0,1,0,0
3815,0,0,0,0,0,0,0,0,0,1,0,0,0,0
3816,0,1,1,0,0,1,0,0,0,1,0,1,0,0


In [215]:
amen_ls = list_df[categorical]['amenities'][0].strip('{}').split(',')
for x in amen_ls:
    x = x.strip('""')
amen_ls

['TV',
 '"Cable TV"',
 'Internet',
 '"Wireless Internet"',
 '"Air Conditioning"',
 'Kitchen',
 'Heating',
 '"Family/Kid Friendly"',
 'Washer',
 'Dryer']

In [221]:
list_df['host_verifications'][4].strip('{}')

"['email', 'phone', 'facebook', 'reviews', 'kba']"

In [234]:
others = [i for i in categorical if i not in list_cols]


,host_location,host_response_time,host_is_superhost,host_has_profile_pic,host_identity_verified,neighbourhood_group_cleansed,property_type,room_type,bed_type,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification
0,"Seattle, Washington, United States",within a few hours,f,t,t,Queen Anne,Apartment,Entire home/apt,Real Bed,f,moderate,f,f
1,"Seattle, Washington, United States",within an hour,t,t,t,Queen Anne,Apartment,Entire home/apt,Real Bed,f,strict,t,t
2,"Seattle, Washington, United States",within a few hours,f,t,t,Queen Anne,House,Entire home/apt,Real Bed,f,strict,f,f
3,"Seattle, Washington, United States",NaN,f,t,t,Queen Anne,Apartment,Entire home/apt,Real Bed,f,flexible,f,f
4,"Seattle, Washington, United States",within an hour,f,t,t,Queen Anne,House,Entire home/apt,Real Bed,f,strict,f,f
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3813,US,within a few hours,f,t,t,Other neighborhoods,House,Entire home/apt,Real Bed,f,strict,f,f
3814,"Seattle, Washington, United States",within an hour,f,t,t,Capitol Hill,Apartment,Entire home/apt,Real Bed,f,moderate,f,f
3815,US,NaN,f,t,f,Rainier Valley,House,Entire home/apt,Real Bed,f,moderate,f,f
3816,"Tacoma, Washington, United States",within an hour,f,t,t,Capitol Hill,Condominium,Entire home/apt,Real Bed,f,moderate,f,f


In [228]:
list_df[quantitative]

,host_since,host_response_rate,host_acceptance_rate,host_listings_count,host_total_listings_count,accommodates,bathrooms,bedrooms,beds,square_feet,...,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,reviews_per_month
0,2011-08-11,96%,100%,3.0,3.0,4,1.0,1.0,1.0,NaN,...,2011-11-01,2016-01-02,95.0,10.0,10.0,10.0,10.0,9.0,10.0,4.07
1,2013-02-21,98%,100%,6.0,6.0,4,1.0,1.0,1.0,NaN,...,2013-08-19,2015-12-29,96.0,10.0,10.0,10.0,10.0,10.0,10.0,1.48
2,2014-06-12,67%,100%,2.0,2.0,11,4.5,5.0,7.0,NaN,...,2014-07-30,2015-09-03,97.0,10.0,10.0,10.0,10.0,10.0,10.0,1.15
3,2013-11-06,NaN,NaN,1.0,1.0,3,1.0,0.0,2.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2011-11-29,100%,NaN,2.0,2.0,6,2.0,3.0,3.0,NaN,...,2012-07-10,2015-10-24,92.0,9.0,9.0,10.0,10.0,9.0,9.0,0.89
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3813,2015-04-13,99%,100%,354.0,354.0,6,2.0,3.0,3.0,NaN,...,2015-09-27,2015-09-27,80.0,8.0,10.0,4.0,8.0,10.0,8.0,0.30
3814,2015-10-14,100%,100%,1.0,1.0,4,1.0,1.0,2.0,NaN,...,2015-12-18,2015-12-24,100.0,10.0,10.0,10.0,10.0,10.0,10.0,2.00
3815,2015-12-30,NaN,NaN,1.0,1.0,2,1.0,1.0,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3816,2015-01-03,100%,NaN,1.0,1.0,2,1.0,0.0,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [318]:
exp_df = list_df[cat_cols]
list_df[list_cols]['host_verifications'][0]

"['email', 'phone', 'reviews', 'kba']"

In [299]:
exp_df['host_verifications'][0]

"['email', 'phone', 'reviews', 'kba']"

In [341]:
target = list_df[list_cols]
target['host_verifications'][0]

"['email', 'phone', 'reviews', 'kba']"

In [342]:
for i in target['host_verifications'].index:
    if not pd.isnull(target['host_verifications'][i]):
        if '{' in target['host_verifications'][i]:
            print('oops')
            target.loc[i, 'host_verifications'] = target['host_verifications'][i].strip('{}').split(',')
        target.loc[i, 'host_verifications'] = ast.literal_eval(target['host_verifications'][i])

/Users/justinharrison/thesis/env/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
/Users/justinharrison/thesis/env/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [350]:
for x in target['host_verifications']:
    print(type(x) )

<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'li

<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'li

<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'li

In [345]:
target['host_verifications'][i]

['email', 'phone', 'reviews', 'kba']